thinking: 
1. why for collaborative filtering:
协同过滤主要是对用户行为和偏好中发现规律，并基于相应的核心策略进行收集和推荐。 用户数据中包含用户行为，商品喜好等方式进行归类和更新。
这一点可以最大程度上对新的“同类型” 用户进行推荐， 不仅在既定用户上的已经做好的分类上进行推荐，当发生更新时，也可以给其他用户提供类似推荐。 但实际上，在解决协同过滤时，（学习过程中） 人为最重要的是消减信息“噪声” ， 并对数据进行归一化， 归一化是为了让信息值上具有相同的“影响力”，减噪视为了用户在使用应用过程中产生的，它可能存在大量的噪音和用户的误操作，可以通过经典的数据挖掘算法过滤掉行为数据中的噪音，这样可以让分析更加精确，对于欧式距离小有了解，其距离可以区分不同类别从而实现其推荐算法的普适度，协同过滤的点子上还可以快速为同类型用户推荐类似但仍然有差异的定制商品或者服务，因此是有必要的。

2. need RS? 
 emmm 通常个人，小公司不需要推荐系统，推荐系统通常被应用于较大的平台型商品综合类型的公司， 首先这类公司具有较大用户base, 信息数据量volumn 值高，从而为了使更多用户具有有特征个性化的推荐，从而实现定制化供给服务。
 
3. 视频打标签问题： 
 对nlp实际上有一定了解，但这个问题问的有一点笼统， 总的来说，视频是一个多媒体的时间序列， 包含图像和音频， 但是nlp的打标签，通常使用的对应 音频的名称进行labeling， 但是忽略了图像时间序列的主要信息包含，可能会出现标签不准确，视频匹配度不高的问题。

In [113]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import SelectKBest,SelectPercentile
import pylab as plt
%matplotlib inline
train_data = pd.read_csv(r'/Users/charles/kkb/bi-attrition-prediction/train.csv')
test_data = pd.read_csv(r'/Users/charles/kkb/bi-attrition-prediction/test.csv')
train_data.head()

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,...,4,80,0,1,4,3,1,0,1,0


In [59]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 35 columns):
user_id                     294 non-null int64
Age                         294 non-null int64
BusinessTravel              294 non-null object
DailyRate                   294 non-null int64
Department                  294 non-null object
DistanceFromHome            294 non-null int64
Education                   294 non-null int64
EducationField              294 non-null object
EmployeeCount               294 non-null int64
EmployeeNumber              294 non-null int64
EnvironmentSatisfaction     294 non-null int64
Gender                      294 non-null object
HourlyRate                  294 non-null int64
JobInvolvement              294 non-null int64
JobLevel                    294 non-null int64
JobRole                     294 non-null object
JobSatisfaction             294 non-null int64
MaritalStatus               294 non-null object
MonthlyIncome               294 non-null in

In [5]:
#check data value whether null.
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 36 columns):
user_id                     1176 non-null int64
Age                         1176 non-null int64
Attrition                   1176 non-null object
BusinessTravel              1176 non-null object
DailyRate                   1176 non-null int64
Department                  1176 non-null object
DistanceFromHome            1176 non-null int64
Education                   1176 non-null int64
EducationField              1176 non-null object
EmployeeCount               1176 non-null int64
EmployeeNumber              1176 non-null int64
EnvironmentSatisfaction     1176 non-null int64
Gender                      1176 non-null object
HourlyRate                  1176 non-null int64
JobInvolvement              1176 non-null int64
JobLevel                    1176 non-null int64
JobRole                     1176 non-null object
JobSatisfaction             1176 non-null int64
MaritalStatus          

In [6]:
#current status
train_data.describe()


,user_id,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,...,1176.000000,1176.0,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.00000,1176.000000,1176.000000
mean,736.124150,36.805272,802.033163,9.159864,2.918367,1.0,1026.960034,2.750850,65.130102,2.724490,...,2.714286,80.0,0.805272,11.161565,2.767007,2.764456,6.982143,4.19898,2.160714,4.098639
std,419.336964,9.065549,405.946729,8.137224,1.009809,0.0,594.763609,1.096221,20.294326,0.715027,...,1.080583,0.0,0.865611,7.747576,1.250756,0.713251,6.094338,3.63124,3.208052,3.564190
min,0.000000,18.000000,104.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
25%,374.750000,30.000000,463.500000,2.000000,2.000000,1.0,498.750000,2.000000,48.000000,2.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.00000,0.000000,2.000000
50%,742.500000,36.000000,805.500000,7.000000,3.000000,1.0,1031.000000,3.000000,65.000000,3.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.00000,1.000000,3.000000
75%,1101.250000,42.250000,1162.000000,14.000000,4.000000,1.0,1555.250000,4.000000,82.250000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.00000,2.250000,7.000000
max,1469.000000,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.00000,15.000000,17.000000


In [30]:
# try seperate the data and lable, normalize the data to numerical classifications
from sklearn.preprocessing import LabelEncoder
train_keys = []
for i in train_data:
    train_keys.append(i)

test_keys = []
for i in test_data:
    test_keys.append(i)

le = LabelEncoder()
var_to_encode_train = train_keys
var_to_encode_test = test_keys

for i in var_to_encode_train:
    train_data[i] = le.fit_transform(train_data[i])

for i in var_to_encode_test:
    test_data[i] = le.fit_transform(test_data[i])




In [101]:
    
labels = train_data["Attrition"]
features = train_data.drop(["Attrition"],axis=1)
features

,user_id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,Travel_Rarely,605,Sales,21,3,Life Sciences,1,1938,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,1546,...,4,80,1,8,3,3,5,4,0,3
2,768,40,Travel_Rarely,300,Sales,26,3,Marketing,1,1066,...,2,80,1,8,3,2,7,7,7,5
3,569,36,Non-Travel,1434,Sales,8,4,Life Sciences,1,789,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Travel_Frequently,599,Sales,24,1,Life Sciences,1,1273,...,4,80,0,1,4,3,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,763,34,Travel_Rarely,1333,Sales,10,4,Life Sciences,1,1055,...,4,80,1,1,2,3,1,1,0,0
1172,835,35,Travel_Rarely,528,Human Resources,8,4,Technical Degree,1,1164,...,2,80,0,6,2,1,5,4,1,4
1173,1216,43,Travel_Rarely,1179,Sales,2,3,Medical,1,1706,...,1,80,1,10,3,3,10,9,8,8
1174,559,38,Travel_Rarely,268,Research & Development,2,5,Medical,1,773,...,2,80,1,6,0,1,1,0,0,1


In [34]:

# use the selector to get the most useable items.
selection = SelectPercentile(percentile=33)
selection.fit(features,labels)
selection.transform(features)
print(selection.get_support())
print(selection.scores_)

features_new = features.loc[:,selection.get_support()]
features_new
test_data_new = test_data.loc[:,selection.get_support()]


[False  True False False False False False False False False False False
 False  True  True False  True  True  True False False False  True False
 False False False  True  True False False  True  True False  True]
[1.03093970e-01 3.74200684e+01 2.94692928e-02 5.05795946e+00
 4.05310924e+00 6.14456958e+00 1.33350415e+00 1.67840804e+00
            nan 1.03093970e-01 1.08348062e+01 1.70432317e+00
 1.42500142e-02 1.55372408e+01 3.61847478e+01 3.05778076e+00
 1.15123502e+01 2.83558039e+01 5.00240170e+01 1.42375594e+00
 1.65823966e+00            nan 6.28526879e+01 1.98308139e+00
 4.32887094e-03 1.43863957e+00            nan 2.01087356e+01
 4.04001972e+01 3.69895595e+00 6.45277762e+00 2.51310770e+01
 3.45263403e+01 2.15919089e+00 3.23776114e+01]


/Users/charles/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 8 21 26] are constant.
  UserWarning)
/Users/charles/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [36]:
from sklearn.model_selection import KFold

def score(clf):

    kf = KFold(n_splits=10,shuffle=True)
    score = 0
    count = 0
    for train_index,test_index in kf.split(features_new):
    #     print("TRAIN",train_index,'\n','TEST',test_index)
        features_train, features_test = features_new.iloc[train_index], features_new.iloc[test_index]
        labels_train, labels_test = labels.iloc[train_index], labels.iloc[test_index]
        
        clf = clf.fit(features_train,labels_train)
        score += clf.score(features_test,labels_test)
        if count == 9:
            mean_score = score/10
        count += 1
    return mean_score



In [37]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=1000)
clf.fit(features_new,labels)
pred = clf.predict(test_data_new)
np.savetxt('result.csv',pred,delimiter=',',fmt='%d',header='result')
score(clf)

0.837621324062002

In [49]:
def modelfit(alg,features=features_new,labels=labels,test_data=test_data_new,useTrain=True,cv=5,early=50):
    if useTrain:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(features.values,label=labels.values)
        xgtest = xgb.DMatrix(test_data.values)
        cvresult = xgb.cv(xgb_param,xgtrain,num_boost_round=alg.get_params()['n_estimators'],nfold=cv,metrics='auc',early_stopping_rounds=early)
        print(cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])

    alg.fit(features,labels,eval_metric='auc')
    
    dtrain_predictions = alg.predict(features)
    dtrain_predprob = alg.predict_proba(features)[:,1]

    print('Model Report')
    print('Accuracy:%.4g' % metrics.accuracy_score(labels.values,dtrain_predictions))
    print('AUC Score(train):%f' % metrics.roc_auc_score(labels,dtrain_predprob))

In [50]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27)
modelfit(xgb1)

36
Model Report
Accuracy:0.9116
AUC Score(train):0.954615


In [104]:
pred = xgb1.predict_proba(test_data_new)
pre=pred[:,1]
len(pre)

294

In [112]:


# test_data.insert(1,"Attrition",pre)
test_data.iloc[:,:2].to_csv('result.csv',index=False)

In [51]:
# test paras just try but not make sense.
param_test1 = {
    'max_depth':range(3,10),
    'min_child_weight':range(0,10)
}
gs1 = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1,n_estimators=50,max_depth=5,
                                min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,
                                objective='binary:logistic',nthread=4,scale_pos_weight=1,seed=27),
                param_grid=param_test1,scoring='roc_auc',n_jobs=4,iid=False,cv=5)
gs1.fit(features,labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=5, min_child_weight=1,
                                     missing=None, n_estimators=50, n_jobs=1,
                                     nthread=4, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=27, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'max_depth': range(3, 10),
                         'min_child_weight': range(0, 10)},
             pre_dispatch='2*n_jobs', refit=True, return_trai